In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,020 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 ht

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import math

In [3]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [4]:
arizona = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Arizona.csv'
alabama = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Alabama.csv'
alaska = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Alaska.csv'
Arkansas = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Arkansas.csv'
California = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/California.csv'
Colorado = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Colorado.csv'
Conneticut = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Conneticut.csv'
Celaware = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Delaware.csv'
Coc = 'https://github.com/bhavya-kashyap/AMC_crosswalk/blob/main/States_raw_data/District_of_Columbia.csv'
Florida = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Florida.csv'
idaho = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Idaho.csv'
illinois = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Illinois.csv'
indiana = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Indiana.csv'
iowa = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Iowa.csv'
kansas = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Kansas.csv'
kentucky = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Kentucky.csv'
states = [Arkansas, California, Colorado, Conneticut, Delaware, Doc, Florida]
state_name = ['Arkansas', "California", "Colorado", "Conneticut", "Delaware", "Doc", "Florida"]

In [26]:
 for i in range(len(states)):
  hospitals = pd.read_csv(states[i])
  hospitals = hospitals[hospitals['AMC_No'].notna()]
  hospitals['AMC_No'] = hospitals["AMC_No"].astype(int)

  baseurl = 'https://freida.ama-assn.org/institution/'
  URLS = []
  for index, i in hospitals.iterrows():
    URLS.append(str(baseurl + str(i['AMC_No'])))
  hospitals['URLS'] = URLS
  len(hospitals)
  def driversetup():
      options = webdriver.ChromeOptions()
      #run Selenium in headless mode
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument("enable-automation")
      #overcome limited resource problems
      options.add_argument('--disable-dev-shm-usage')
      options.add_argument("lang=en")
      #open Browser in maximized mode
      options.add_argument("start-maximized")
      #disable infobars
      options.add_argument("disable-infobars")
      #disable extension
      options.add_argument("--disable-browser-side-navigation")
      options.add_argument("--disable-gpu")
      options.add_argument("--disable-extensions")
      options.add_argument("--incognito")
      options.add_argument("--disable-blink-features=AutomationControlled")
      
      driver = webdriver.Chrome(options=options)

      driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

      return driver

  def pagesource(url, driver):
      driver = driver
      driver.get(url)
      time.sleep(12)
      soup = BeautifulSoup(driver.page_source)
      #driver.close()
      return soup
  
  mc = []
  grad = []
  mcl = []
  add = []
  d = driversetup()
  for index, i in hospitals.iterrows():
    url = i['URLS']
    s = pagesource(url, d)
    med_school = list(s.findAll('small', class_="bold"))
    address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
    if len(med_school) == 0:
      mc.append("")
      mcl.append("")
      grad.append("")
    elif len(med_school) == 1:
      a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
      name = a[0].findAll('small', class_='bold')[0].text
      relationship = a[0].findAll('div', class_='')[0].text
      if 'Limited' in relationship:
        mcl.append(name)
        mc.append("")
        grad.append("")
      elif 'Major' in relationship:
        mc.append(name)
        mcl.append("")
        grad.append("")
      elif 'Graduate' in relationship:
        mc.append("")
        mcl.append("")
        grad.append(name)
      else:
        print("NEW RELATIONSHIP FOUND : ", name)
        break
    elif len(med_school) > 1:   
        major = []
        limited = []
        graduate = []
        a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
        for i in range(len(a)):
          name = a[i].findAll('small', class_='bold')[0].text
          relationship = a[i].findAll('div', class_='')[0].text
          if 'Limited' in relationship:
            limited.append(name)
          elif 'Graduate' in relationship:
            graduate.append(name)
          elif 'Major' in relationship:
            major.append(name)
          else:
            print("New relationship found")
            break 
        if major == []:
          major = ''
        if limited == []:
          limited = ''
        if grad == []:
          grad = ''
        mc.append(major)
        mcl.append(limited)
        grad.append(graduate)
    if len(address) != 0:
      address[0].br.string = ', '
      add.append(address[0].text)
    else:
      add.append("")
    print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
    if len(mc) > (index+1):
      break
    
  print(len(mc),len(mcl),len(grad),len(add))
  hospitals['Affiliation_Major'] = mc
  hospitals['Affiliation_Limited'] = mcl
  hospitals['Affiliation_Graduate'] = grad
  hospitals['Medical_Center_Address'] = add 
  hospitals.to_csv('{}_crosswalk.csv').format(state_name[i])



In [27]:
len(hospitals)

113

In [21]:
# def driversetup():
#     options = webdriver.ChromeOptions()
#     #run Selenium in headless mode
#     options.add_argument('--headless')
#     options.add_argument('--no-sandbox')
#     options.add_argument("enable-automation")
#     #overcome limited resource problems
#     options.add_argument('--disable-dev-shm-usage')
#     options.add_argument("lang=en")
#     #open Browser in maximized mode
#     options.add_argument("start-maximized")
#     #disable infobars
#     options.add_argument("disable-infobars")
#     #disable extension
#     options.add_argument("--disable-browser-side-navigation")
#     options.add_argument("--disable-gpu")
#     options.add_argument("--disable-extensions")
#     options.add_argument("--incognito")
#     options.add_argument("--disable-blink-features=AutomationControlled")
    
#     driver = webdriver.Chrome(options=options)

#     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

#     return driver

# def pagesource(url, driver):
#     driver = driver
#     driver.get(url)
#     time.sleep(12)
#     soup = BeautifulSoup(driver.page_source)
#     #driver.close()
#     return soup



In [ ]:
# d = driversetup()
# relationship = []
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   print(i.Med_center, index, relationship, len(med_school))
#   for i in range(len(med_school)):
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    
#     for i in range(len(a)):
#       rel = a[i].findAll('div', class_='')[0].text
#       if rel not in relationship:
#         print(med_school[i].text, rel)
#         relationship.append(rel)
# relationship

In [28]:
# mc = []
# grad = []
# mcl = []
# add = []
# d = driversetup()
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
#   if len(med_school) == 0:
#     mc.append("")
#     mcl.append("")
#     grad.append("")
#   elif len(med_school) == 1:
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
#     name = a[0].findAll('small', class_='bold')[0].text
#     relationship = a[0].findAll('div', class_='')[0].text
#     if 'Limited' in relationship:
#       mcl.append(name)
#       mc.append("")
#       grad.append("")
#     elif 'Major' in relationship:
#       mc.append(name)
#       mcl.append("")
#       grad.append("")
#     elif 'Graduate' in relationship:
#       mc.append("")
#       mcl.append("")
#       grad.append(name)
#     else:
#       print("NEW RELATIONSHIP FOUND : ", name)
#       break
#   elif len(med_school) > 1:   
#       major = []
#       limited = []
#       graduate = []
#       a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
#       for i in range(len(a)):
#         name = a[i].findAll('small', class_='bold')[0].text
#         relationship = a[i].findAll('div', class_='')[0].text
#         if 'Limited' in relationship:
#           limited.append(name)
#         elif 'Graduate' in relationship:
#           graduate.append(name)
#         elif 'Major' in relationship:
#           major.append(name)
#         else:
#           print("New relationship found")
#           break 
#       if major == []:
#         major = ''
#       if limited == []:
#         limited = ''
#       if grad == []:
#         grad = ''
#       mc.append(major)
#       mcl.append(limited)
#       grad.append(graduate)
#   if len(address) != 0:
#     address[0].br.string = ', '
#     add.append(address[0].text)
#   else:
#     add.append("")
#   print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
#   if len(mc) > (index+1):
#     break



0 mc 1  mcl 1  grad 1  add 1 
1 mc 2  mcl 2  grad 2  add 2 
2 mc 3  mcl 3  grad 3  add 3 
3 mc 4  mcl 4  grad 4  add 4 
4 mc 5  mcl 5  grad 5  add 5 
5 mc 6  mcl 6  grad 6  add 6 
6 mc 7  mcl 7  grad 7  add 7 
7 mc 8  mcl 8  grad 8  add 8 
8 mc 9  mcl 9  grad 9  add 9 
9 mc 10  mcl 10  grad 10  add 10 
10 mc 11  mcl 11  grad 11  add 11 
11 mc 12  mcl 12  grad 12  add 12 
12 mc 13  mcl 13  grad 13  add 13 
13 mc 14  mcl 14  grad 14  add 14 
14 mc 15  mcl 15  grad 15  add 15 
15 mc 16  mcl 16  grad 16  add 16 
16 mc 17  mcl 17  grad 17  add 17 
17 mc 18  mcl 18  grad 18  add 18 
18 mc 19  mcl 19  grad 19  add 19 
19 mc 20  mcl 20  grad 20  add 20 
20 mc 21  mcl 21  grad 21  add 21 
21 mc 22  mcl 22  grad 22  add 22 
22 mc 23  mcl 23  grad 23  add 23 
23 mc 24  mcl 24  grad 24  add 24 
24 mc 25  mcl 25  grad 25  add 25 
25 mc 26  mcl 26  grad 26  add 26 
26 mc 27  mcl 27  grad 27  add 27 
27 mc 28  mcl 28  grad 28  add 28 
28 mc 29  mcl 29  grad 29  add 29 
29 mc 30  mcl 30  grad 30  add 

In [23]:
# print(len(mc),len(mcl),len(grad),len(add))
# hospitals['Affiliation_Major'] = mc
# hospitals['Affiliation_Limited'] = mcl
# hospitals['Affiliation_Graduate'] = grad
# hospitals['Medical_Center_Address'] = add

9 9 9 9


In [25]:
# hospitals.to_csv('Alaska_crosswalk.csv')